# Introduction

The objective here is to do some unit testing on every function and blocks of the EMANN method.
Moreover this we serve as example code of how to do things and use the functions + reminder of why I have coded this way.


- [1. Data](#Load-datasets)
    - [1.1 Loading of datasets](#Load-datasets)
    - [1.2 Transformation of datasets](#Transform-datasets)
- [2. Clusters](#Clusters)
- [3. Optimal transport](#Optimal-Transport)
    - [3.1 Optimal transport solver](#Optimal-Transport)
    - [3.2 Align](#Align)
    - [3.3 Cost matrix](#Cost-matrix)
- [4. Neural Networks](#Neural-Networks)
    - [4.1 Special blocks](#Special-blocks)
    - [4.2 Compiler](#Compiler)
    - [4.3 CNN class](#Neural-Network-class)
- [5. Examples](#Examples)
    - [5.1 Classification](#Classification)
    - [5.2 Regression](#Regression)
    - [5.3 MNIST](#MNIST)
    - [5.4 DANN](#DANN)
    - [5.5 Dual proba classification](#Dual-proba-classification)
    - [5.6 Bi classification](#Bi-classification)
    - [5.7 One loop EMANN](#One-loop-EMANN)
- [6. ReGenerate Data](#ReGenerate-Data)


In [ ]:
from __future__ import division, print_function
import sys
if '..' not in sys.path:
    sys.path.append('..')

import theano
import theano.tensor as T
import lasagne

import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from sklearn.metrics import confusion_matrix

In [ ]:
%matplotlib inline

In [ ]:
import visual


# Load datasets

How to load/generate and build Datasets. 

[**[Back to top]**](#Introduction)

## Datasets Imports 

In [ ]:
from datasets.toys import make_clouds, make_circles, make_X, make_moons
from datasets.mnist import load_mnist
from datasets.utils import make_dataset


# Transform datasets


[**[Back to top]**](#Introduction)

## Transformation Imports

In [ ]:
from datasets.utils import make_domain_dataset, make_corrector_dataset
from datasets import transform

# Clusters

Here we test and illustrate the use of sci-kit learn's KMeans in our problem (Optimal transport).

[**[Back to top]**](#Introduction)

In [ ]:
X_src, y_src = make_clouds(n_samples=50, n_classes=6)
X_tgt, y_tgt = make_moons(n_samples=500)
data_name = "Clouds-to-moons"

In [ ]:
from sklearn.cluster import KMeans

k_src = 10
k_tgt = 11
# We do not need to have the same number of cluster in the source and target data.
k_means_src = KMeans(n_clusters=k_src).fit(X_src)
k_means_tgt = KMeans(n_clusters=k_tgt).fit(X_tgt)


In [ ]:
def mass(k_means):
    """
    Params
    ------
        k_means: (sklearn.cluster.KMeans instance) should be trained
    Return
    ------
        w: (numpy.array [n_clusters]) the mass of each clusters 
    """
    w = np.unique(k_means.labels_, return_counts=True)[1]
    w = w/np.sum(w)
    return w
    

For now the *mass computation* is very simple. Maybe we can improve the results by taking labels or other things into account.

In [ ]:
w_src = mass(k_means_src)
# w_src = np.abs(np.sin(-w_src-np.arange(w_src.shape[0])))
# w_src /= np.sum(w_src)

w_tgt = mass(k_means_tgt)
# I modify the mass in order to test a custom final distrib of the data in the coupling dataset
w_tgt = np.abs(np.sin(-w_tgt-0.6*np.arange(w_tgt.shape[0])))
# w_tgt = np.exp(-w_tgt-np.arange(w_tgt.shape[0]))
w_tgt /= np.sum(w_tgt)
cost_mat = np.random.uniform(0,1, size=(w_src.shape[0], w_tgt.shape[0]))


In [ ]:
visual.mat(cost_mat)
plt.show()

Now every thing is set up(masses and cost matrix). We can solve the optimal transport problem.

# Optimal Transport

[**[Back to top]**](#Introduction)

In [ ]:
# Import the solvers functions
from opt_transport import opt_transp_sup, computeTransportSinkhorn, computeTransportSinkhornLabelsLpL1

In [ ]:
# transp = opt_transp_sup(k_means_src.cluster_centers_, k_means_tgt.cluster_centers_)
transp = computeTransportSinkhorn(w_src, w_tgt, cost_mat, reg=10)

In [ ]:
visual.mat(transp)
plt.show()

This is very easy.

TODO : Add more test and visualisation of the different transport solvers.

# Align

[**[Back to top]**](#Introduction)

Here we build the datasets that will be fed to the NN thanks to the transport matrix.

In [ ]:
print(transp.shape, X_src.shape, k_means_src.labels_.shape)

In [ ]:
from align_learn.preprocess import align

Sanity check. The cluster distribution of the **aligned data** should be the same as the **target distrib**

In [ ]:
align_idx, cluster_T = align(transp, k_means_src.labels_, k_means_tgt.labels_)
# print(np.unique(res).shape, X_src.shape, X_tgt.shape)
uniq, count = np.unique(cluster_T, return_counts=True)
# plt.plot(np.sum(transp,0), label='transp.sum(0)')
plt.plot(w_tgt, label='w_tgt')
# plt.plot(np.sum(transp,1), label='transp.sum(1)')
plt.plot(w_src, label='w_src')
plt.plot(count/cluster_T.shape[0], label='mapping')
plt.legend(bbox_to_anchor=(1.25, 1.))
plt.show()

In [ ]:
X_S, y_S = X_src, y_src
X_T, y_T = X_tgt[align_idx], y_tgt[align_idx]

We can see in the next cell that there is a big hole in the coupling dataset. This is due to the target weights set near zero for one of the cluster.

In [ ]:
visual.target_2D(X_T, y_T)
visual.target_2D(X_tgt, y_tgt)
plt.show()

In [ ]:
X_T.shape, X_S.shape

Build the probabilities to be predict by the NN.

In [ ]:
def train_dataset(X_src, X_tgt, k_means_src, k_means_tgt, transp):
    """
    """
    align_idx, cluster_T = align(transp, k_means_src.labels_, k_means_tgt.labels_)
    X_S, y_S = X_src, y_src
    X_T, y_T = X_tgt[align_idx], y_tgt[align_idx]
    # Build the probabilities to be predict
    # For the source data
    proba_src = np.zeros((X_S.shape[0], k_means_src.n_clusters))
    proba_src[np.arange(X_S.shape[0]), k_means_src.labels_] = 1.
    proba_tgt = transp[k_means_src.labels_]
    proba_tgt = proba_tgt / np.sum(proba_tgt, 1).reshape(-1, 1)
    Y_S = np.hstack([proba_src, proba_tgt])
    
    # Build the probabilities to be predict
    # For the aligned target data
    proba_tgt = np.zeros((X_T.shape[0], k_means_tgt.n_clusters))
    proba_tgt[np.arange(X_T.shape[0]), cluster_T] = 1.
    proba_src = transp[:, cluster_T].T
    proba_src = proba_src / np.sum(proba_src, 1).reshape(-1, 1)
    Y_T = np.hstack([proba_src, proba_tgt])
    
    Y = np.vstack([Y_S, Y_T])
    X = np.vstack([X_S, X_T])
    return X, Y

In [ ]:
X, Y = train_dataset(X_src, X_tgt, k_means_src, k_means_tgt, transp)

In [ ]:
print(Y[0])
print(Y[350])
print(np.sum(Y[350]))

In [ ]:
X.shape, Y.shape

# Cost matrix

[**[Back to top]**](#Introduction)

# Neural Networks

Here will be tested and illustrated the neural network building, training and monitoring from the simple classification to an alternative training on a multiple output neural network.

4 steps :
1. Build the architecture (symbolic computation graph) with lasagne
2. Compile the NN functions (train, valid, output, prediction, etc)
3. Do forward backward training loop
4. Plot the statistics and test the NN

In [ ]:
import theano
import theano.tensor as T
import lasagne

import nn.compilers as nnc
import nn.block as nnb

from nn.helper import CNN, NN
from nn.rgl import ReverseGradientLayer
from logs import log_fname, new_logger, empty_logger


# Special blocks

Here we build the layers and introduce some function to build some special/frequently used structures.

[**[Back to top]**](#Introduction)

In [ ]:
# Set the hyper parameters
# ========================
hp_lambda = 0.
batchsize = 20

# Learning rates and momentums
label_rate = 0.1
label_mom = 0.9

domain_rate = 0.1
domain_mom = 0.9

# Get a logger
logger = new_logger()


In [ ]:
# Compute some data dependant params
# ==================================
_shape = np.shape(X)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.matrix('targets')

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))


In [ ]:
# Build the layers
# ================
input_layer = lasagne.layers.InputLayer(shape=shape)

dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=5,
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
dense_2 = lasagne.layers.DenseLayer(
                dense_1,
                num_units=5,
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
dense_3 = lasagne.layers.DenseLayer(
                dense_2,
                num_units=5,
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )


In [ ]:
# Or we could use the dense block function
dense_3 = nnb.dense(input_layer, [5,5,5], activation='relu')

Now here is the way to get the inputs variables. Even in a complex structure the input must be a InputLayer.

In [ ]:
concat_test = lasagne.layers.ConcatLayer([dense_1, dense_2, dense_3], axis=1)
input_2 = lasagne.layers.InputLayer((20,2))
concat_test_2 = lasagne.layers.ConcatLayer([concat_test, input_2], axis=1)
dense_final = lasagne.layers.DenseLayer(concat_test_2, 5)
lasagne.layers.get_all_layers(dense_final)

In [ ]:
from nn.compilers import get_input_vars

Sanity check

In [ ]:
input_variables = get_input_vars(dense_final)
print(input_variables)
print(input_variables[0] is input_variables[1])
print(input_variables[0] == input_variables[1])

# Compiler
Append the last part and compile

[**[Back to top]**](#Introduction)

In [ ]:
# Get the output layer :
# ======================
predict_layer = nnb.multi_proba(dense_1, [k_src, k_tgt])

# Instanciate the NN :
# ====================
nn = NN(predict_layer, name='Special proba test')

# Compile :
# =========
# nn.compile(compiler, lr=label_rate)
nn.compile(nnc.crossentropy_sgd_mom, lr=label_rate, mom=label_mom, target_var=target_var,
           regularization=None, reg_param=.1)
nn.compile(nnc.crossentropy_validation, target_var=target_var)
nn.compile(nnc.output)



# Training

[**[Back to top]**](#Introduction)

In [ ]:
data = make_dataset(X, Y, batchsize=batchsize) # Those X and Y are the coupling dataset.

In [ ]:
nn.train(data, num_epochs=100)
final_stats = nn.global_stats

# Visualization of the learning procedure

[**[Back to top]**](#Introduction)

In [ ]:
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(final_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');


# Neural Network class

[**[Back to top]**](#Introduction)

Some sanity check on the working of AttributeDict.

In [ ]:
nn = CNN()
nn['out'] = 'layer'
def foo(string, **kwargs):
    def bar():
        print('compile('+', '.join([string]+kwargs.keys())+')')
        print('OK')
    return {'bar': bar}
nn.compile('out', foo, kwargs1=8, kwargs2='bla')
# TEST
nn['out'].bar()
nn['out']['bar']()
nn.parts.out.bar()

# Examples

Let's have some example code with the complete workflow.

# Classification

[**[Back to top]**](#Introduction)

Get the data

In [ ]:
n_classes = 5
n_samples = 1000
test_dataset = make_dataset(*make_clouds(n_samples=n_samples, n_classes=n_classes), batchsize=60)
# test_dataset = make_dataset(*make_moons(n_samples=n_samples), batchsize=60)
print('Dataset contains :', test_dataset.keys())

Build the NN

In [ ]:
# Get general information :
# =========================
X = test_dataset.X_train
_shape = np.shape(X)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.ivector('targets')

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape)

dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=3,
                nonlinearity=lasagne.nonlinearities.tanh,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
softmax_layer = lasagne.layers.DenseLayer(
                dense_1,
                num_units=n_classes,
                nonlinearity=lasagne.nonlinearities.softmax,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )

# Instanciate the NN :
# ====================

# nn = CNN(name='Clouds test')
nn = NN(softmax_layer, name='Clouds test')
# nn.add_output('main', softmax_layer)

# Compile :
# =========
nn.compile(nnc.crossentropy_sgd_mom, lr=label_rate, mom=label_mom, target_var=target_var,
           regularization=None, reg_param=.1)
nn.compile(nnc.crossentropy_validation, target_var=target_var)
nn.compile(nnc.output)




Train the NN

In [ ]:
# Train the nn :
# ==============
# nn.train([test_dataset, ], ['main', ]);
nn.train(test_dataset);


Look at the results

In [ ]:
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');


In [ ]:
for c in range(len(np.unique(test_dataset.y_test))):
    visual.bound(test_dataset.X_test, test_dataset.y_test, nn.funs.output, class_idx=c);
plt.show()

# Regression

[**[Back to top]**](#Introduction)

# MNIST

Here is illustrated the use of convolution layers for classification.

In [ ]:
batchsize = 1000
n_classes = 10
mnist_dataset = make_dataset(*load_mnist(), batchsize=batchsize)

[**[Back to top]**](#Introduction)

In [ ]:
plt.imshow(mnist_dataset['X_test'][60].reshape(28,28))
plt.show()

In [ ]:
# Get general information :
# =========================
_shape = np.shape(mnist_dataset['X_train'])
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.ivector('targets')

# Logs
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape)

dropout_1 = lasagne.layers.DropoutLayer(input_layer)
conv_1 = lasagne.layers.Conv2DLayer(
                dropout_1, 5, (5,5),
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
pool_1 = lasagne.layers.MaxPool2DLayer(conv_1, (2,2))
dropout_2 = lasagne.layers.DropoutLayer(pool_1)
conv_2 = lasagne.layers.Conv2DLayer(
                dropout_2, 5, (3,3),
                nonlinearity=lasagne.nonlinearities.rectify,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
pool_2 = lasagne.layers.MaxPool2DLayer(conv_2, (2,2))
dense_1 = lasagne.layers.DenseLayer(pool_2, 30)
softmax_layer = lasagne.layers.DenseLayer(
                dense_1,
                num_units=n_classes,
                nonlinearity=lasagne.nonlinearities.softmax,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )

# Instanciate the NN :
# ====================
nn = NN(softmax_layer, name='MNIST test')

# Compile :
# =========
nn.compile(nnc.classification_sgd_mom, lr=0.01, mom=0.9, regularization=None, reg_param=.1)
nn.compile(nnc.classification_validation)
nn.compile(nnc.output)


In [ ]:
# Train the nn :
# ==============
nn.train(mnist_dataset, num_epochs=40);

In [ ]:
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='acc')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');

## DANN

[**[Back to top]**](#Introduction)

### Datasets

In [ ]:
n_classes = 2
n_samples = 1000
# X_src, y_src = make_clouds(n_samples=n_samples, n_classes=n_classes)
X_src, y_src = make_moons(n_samples=n_samples)
src_dataset = make_dataset(X_src, y_src, batchsize=60)

X_tgt, y_tgt = transform.rotate(X_src, y_src, angle=-45.)
tgt_dataset =  make_dataset(X_tgt, y_tgt, batchsize=60)

domain_dataset = make_domain_dataset([src_dataset, tgt_dataset])


### NN

In [ ]:
from nn.clone import clone_layer

In [ ]:
# Get general information :
# =========================
X = src_dataset.X_train
batchsize = src_dataset.batchsize
_shape = np.shape(X)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.ivector('targets')

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape)

dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=3,
                nonlinearity=lasagne.nonlinearities.tanh,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )
softmax_layer = lasagne.layers.DenseLayer(
                dense_1,
                num_units=n_classes,
                nonlinearity=lasagne.nonlinearities.softmax,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )

In [ ]:
# Instanciate the NN :
# ====================

nn = CNN(name='Moons test')
nn.add_output('main', softmax_layer)
nn.add_output('adversarial', [dense_1, 
                              clone_layer(dense_1, input_layer=lasagne.layers.InputLayer(shape=shape))])

# Compile :
# =========
nn.compile('main', nnc.classification_sgd_mom, lr=0.1)
nn.compile('main', nnc.classification_validation)
nn.compile('main', nnc.output)
nn.compile('adversarial', nnc.adversarial, lr=0.1, hp_lambda=0.8)

# Train the nn :
# ==============
nn.train([src_dataset, domain_dataset], ['main', 'adversarial'], num_epochs=100);
# nn.train([src_dataset,], ['main',], num_epochs=60);
# nn.train([domain_dataset,], ['adversarial',], num_epochs=60);

# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');
fig, ax = visual.learning_curve(nn.global_stats, regex='acc')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');


# Boundary
# ========
visual.bound(src_dataset.X_test, src_dataset.y_test, nn['main'].output);
plt.show()
visual.bound(tgt_dataset.X_test, tgt_dataset.y_test, nn['main'].output);
plt.show()

loss, acc = nn.parts['main'].valid(src_dataset.X_test, src_dataset.y_test)
print('Source : loss = {}, acc = {}'.format(loss, acc))
loss, acc = nn.parts['main'].valid(tgt_dataset.X_test, tgt_dataset.y_test)
print('Target : loss = {}, acc = {}'.format(loss, acc))

In [ ]:
# Instanciate the NN :
# ====================

nn = CNN(name='Moons test')
nn.add_output('main', softmax_layer)
nn.add_output('adversarial', [dense_1, 
                              clone_layer(dense_1, input_layer=lasagne.layers.InputLayer(shape=shape))])

# Compile :
# =========
nn.compile('main', nnc.classification_sgd_mom, lr=0.1)
nn.compile('main', nnc.classification_validation)
nn.compile('main', nnc.output)

# Train the nn :
# ==============
nn.train([src_dataset], ['main'], num_epochs=100);

# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');
fig, ax = visual.learning_curve(nn.global_stats, regex='acc')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
# visual.learning_curve(final_stats, regex='domain.* acc');

# Boundary
# ========
visual.bound(src_dataset.X_test, src_dataset.y_test, nn['main'].output);
plt.show()
visual.bound(tgt_dataset.X_test, tgt_dataset.y_test, nn['main'].output);
plt.show()

loss, acc = nn.parts['main'].valid(src_dataset.X_test, src_dataset.y_test)
print('Source : loss = {}, acc = {}'.format(loss, acc))
loss, acc = nn.parts['main'].valid(tgt_dataset.X_test, tgt_dataset.y_test)
print('Target : loss = {}, acc = {}'.format(loss, acc))

# Dual proba classification

[**[Back to top]**](#Introduction)

In [ ]:
from sklearn.preprocessing import LabelBinarizer

n_classes = 3
n_samples = 1000
X, y = make_clouds(n_samples=n_samples, n_classes=n_classes)
# X_src, y_src = make_moons(n_samples=n_samples)
lb = LabelBinarizer()
y_lb = lb.fit_transform(y)
y_c = np.concatenate([y_lb, y_lb], axis=1)
dataset = make_dataset(X, np.array(y_c, dtype=np.float32), batchsize=60)


In [ ]:
print(y.shape, X.shape, y_c.shape)

In [ ]:
# Get general information :
# =========================
batchsize = dataset.batchsize
_shape = np.shape(dataset.X_train)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.ivector('targets')

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape)

dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=3,
                nonlinearity=lasagne.nonlinearities.sigmoid,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )


end_layer = nnb.multi_proba(dense_1, [n_classes, n_classes])


In [ ]:
# Instanciate the NN :
# ====================
nn = NN(end_layer, name='Dual test')

# Compile :
# =========
nn.compile(nnc.crossentropy_sgd_mom, lr=0.1, mom=0.9)
nn.compile(nnc.crossentropy_validation)
nn.compile(nnc.output)

# Train the nn :
# ==============
nn.train(dataset, num_epochs=100);


In [ ]:
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()


In [ ]:
y_pred = nn.funs.output(dataset.X_test)[0]
i = np.random.randint(0, dataset.X_test.shape[0])
# print('\n'.join('{:1.5f}--{:1.5f}'.format(pred, truth) for pred, truth in zip(y_pred[i], data.y_test[i])))
width=0.4
plt.bar(np.arange(n_classes+n_classes), y_pred[i], width, color='r')
plt.bar(np.arange(n_classes+n_classes)+width, dataset.y_test[i], width, color='b')
plt.title("One point distrib")
# plt.yscale('log')
plt.show()


In [ ]:
dataset2 = make_dataset(X, y, batchsize=60)
# Instanciate the NN :
# ====================

nn = CNN(name='Dual test')
nn.add_output('01', lasagne.layers.DenseLayer(dense_1, n_classes, nonlinearity=lasagne.nonlinearities.softmax))
nn.add_output('02', lasagne.layers.DenseLayer(dense_1, n_classes, nonlinearity=lasagne.nonlinearities.softmax))

# Compile :
# =========
nn.compile('01', nnc.classification_sgd_mom, lr=0.1)
nn.compile('02', nnc.classification_sgd_mom, lr=0.1)
nn.compile('01', nnc.classification_validation)
nn.compile('02', nnc.classification_validation)
nn.compile('01', nnc.output)
nn.compile('02', nnc.output)

# Train the nn :
# ==============
nn.train([dataset2, dataset2], ['01', '02'], num_epochs=100);


In [ ]:
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()


# Bi classification

[**[Back to top]**](#Introduction)

In [ ]:
n_classes_1 = 4
n_classes_2 = 4
n_samples = 1000
X_1, y_1 = make_clouds(n_samples=n_samples, n_classes=n_classes_1)
X_2, y_2 = make_circles(n_samples=n_samples,  n_classes=n_classes_2)
dataset_1 = make_dataset(X_1, y_1, batchsize=60)
dataset_2 = make_dataset(X_2, y_2, batchsize=60)
data_name='Clouds + Circles'

In [ ]:
# Get general information :
# =========================
batchsize = dataset.batchsize
_shape = np.shape(dataset.X_train)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.ivector('targets')

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
# Build the layers
input_layer = lasagne.layers.InputLayer(shape=shape)

dense_1 = lasagne.layers.DenseLayer(
                input_layer,
                num_units=3,
                nonlinearity=lasagne.nonlinearities.sigmoid,
                # W=lasagne.init.Uniform(range=0.01, std=None, mean=0.0),
                )

# Instanciate the NN :
# ====================

nn = CNN(name='Dual test')
nn.add_output('01', lasagne.layers.DenseLayer(dense_1, n_classes_1, nonlinearity=lasagne.nonlinearities.softmax))
nn.add_output('02', lasagne.layers.DenseLayer(dense_1, n_classes_2, nonlinearity=lasagne.nonlinearities.softmax))

# Compile :
# =========
nn.compile('01', nnc.classification_sgd_mom, lr=0.1)
nn.compile('02', nnc.classification_sgd_mom, lr=0.1)
nn.compile('01', nnc.classification_validation)
nn.compile('02', nnc.classification_validation)
nn.compile('01', nnc.output)
nn.compile('02', nnc.output)

# Train the nn :
# ==============
nn.train([dataset_1, dataset_2], ['01', '02'], num_epochs=100);

# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()
fig, ax = visual.learning_curve(nn.global_stats, regex='acc')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()


In [ ]:
for c in range(len(np.unique(dataset_1.y_test))):
    visual.bound(dataset_1.X_test, dataset_1.y_test, nn.parts['01'].output, class_idx=c);
plt.show()
for c in range(len(np.unique(dataset_2.y_test))):
    visual.bound(dataset_2.X_test, dataset_2.y_test, nn.parts['02'].output, class_idx=c);
plt.show()

# One loop EMANN

[**[Back to top]**](#Introduction)

In [ ]:
EM_ITER = 0

**Generate data**

In [ ]:
n_classes_1 = 4
n_classes_2 = 4
n_samples = 1000
X_src, y_src = make_clouds(n_samples=n_samples, n_classes=n_classes_1)

X_tgt, y_tgt = X_src, y_src
# X_tgt, y_tgt = make_circles(n_samples=n_samples,  n_classes=n_classes_2)

data_name='Clouds -> Same'

**Clusters**

In [ ]:
from sklearn.cluster import KMeans

k_src = 10
k_tgt = 10
# We do not need to have the same number of cluster in the source and target data.
k_means_src = KMeans(n_clusters=k_src).fit(X_src)
k_means_tgt = KMeans(n_clusters=k_tgt).fit(X_tgt)
# Mass and cost matrix
w_src = mass(k_means_src)
w_tgt = mass(k_means_tgt)
cost_mat = np.random.uniform(0,1, size=(w_src.shape[0], w_tgt.shape[0]))
visual.mat(cost_mat)
plt.title("Cost matrix")
plt.show()

**Optimal Transport**

In [ ]:
transp = computeTransportSinkhorn(w_src, w_tgt, cost_mat, reg=10)
visual.mat(transp)
plt.title("Transport matrix")
plt.show()

**Dual Proba dataset**

In [ ]:
X, Y = train_dataset(X_src, X_tgt, k_means_src, k_means_tgt, transp)
data = make_dataset(X, Y, batchsize=100)

**Neural Network Architecture**

In [ ]:
# Get general information :
# =========================
batchsize = data.batchsize
_shape = np.shape(data.X_train)
n_dim = len(_shape)
n_features = np.prod(_shape[1:])

shape = (batchsize,) + _shape[1:]
target_var = T.ivector('targets')

# Logs
logger.info('Building the input and output variables for : {}'.format(data_name))
logger.info('Input data expected shape : {}'.format(shape))

# Build the layers :
# ==================
def build():
    input_layer = lasagne.layers.InputLayer(shape=shape)

    dense_1 = lasagne.layers.DenseLayer(input_layer, 3, nonlinearity=lasagne.nonlinearities.sigmoid)
    dense_2 = lasagne.layers.DenseLayer(dense_1, 3, nonlinearity=lasagne.nonlinearities.sigmoid)

    # With concat :
    cluster_src = lasagne.layers.DenseLayer(dense_2, k_src, nonlinearity=lasagne.nonlinearities.softmax)
    cluster_tgt = lasagne.layers.DenseLayer(dense_2, k_tgt, nonlinearity=lasagne.nonlinearities.softmax)
    concat_layer = lasagne.layers.ConcatLayer([cluster_src, cluster_tgt], axis=1)

    end_layer = concat_layer
    return end_layer, dense_2
end_layer, repr_layer = build()

**Compile the NN**

In [ ]:
# Instanciate the NN :
# ====================
# nn = NN(end_layer, name='EMANN test')
nn = CNN(name='EMANN test')
nn.add_output('proba', end_layer)
nn.add_output('repr', repr_layer)

# Compile :
# =========
# nn.compile(nnc.crossentropy_sgd_mom, lr=0.1, mom=0.9)
# nn.compile(nnc.crossentropy_validation)
# nn.compile(nnc.output)

nn.compile('proba', nnc.crossentropy_sgd_mom, lr=0.1, mom=0.9)
nn.compile('proba', nnc.crossentropy_validation)
nn.compile('proba', nnc.output)
nn.compile('repr', nnc.output)


**Train the NN**

In [ ]:
# Train the nn :
# ==============
# nn.train(data, num_epochs=100);
nn.train([data], ['proba'], num_epochs=100);
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()

In [ ]:
y_pred = nn.parts['proba'].output(data.X_test)[0]
i = np.random.randint(0, data.X_test.shape[0])
# print('\n'.join('{:1.5f}--{:1.5f}'.format(pred, truth) for pred, truth in zip(y_pred[i], data.y_test[i])))
width=0.4
plt.bar(np.arange(k_src+k_tgt), y_pred[i], width, color='r')
plt.bar(np.arange(k_src+k_tgt)+width, data.y_test[i], width, color='b')
plt.title("One point distrib")
plt.legend(bbox_to_anchor=(1.2,1.))
# plt.yscale('log')
plt.show()

## **EM Starts Here !**

**Clusters**

In [ ]:
from sklearn.metrics import pairwise_distances
# X_src_repr = nn.parts['repr'].output(X_src)
# X_tgt_repr = nn.parts['repr'].output(X_tgt)
centers_src = nn.parts['repr'].output(k_means_src.cluster_centers_)[0]
centers_tgt = nn.parts['repr'].output(k_means_tgt.cluster_centers_)[0]
cost_mat = pairwise_distances(centers_src, centers_tgt)
cost_mat = cost_mat**2
visual.mat(cost_mat)
plt.title("Cost matrix")
plt.show()

**Optimal Transport**

In [ ]:
transp = computeTransportSinkhorn(w_src, w_tgt, cost_mat, reg=5)
visual.mat(transp)
plt.title("Transport matrix")
plt.show()

**Dual Proba dataset**

In [ ]:
X, Y = train_dataset(X_src, X_tgt, k_means_src, k_means_tgt, transp)
data = make_dataset(X, Y, batchsize=100)

**Neural network** (re-initialization)

**Train the NN**

In [ ]:
# Train the nn :
# ==============
# nn.train(data, num_epochs=100);
nn.train([data], ['proba'], num_epochs=100);
# ================
# Learning curve
# ================
fig, ax = visual.learning_curve(nn.global_stats, regex='loss')
#     SAVE
# fig.tight_layout()
# fig.savefig(fig_title+'-Learning_curve.png',bbox_inches='tight')
fig.show()

EM_ITER += 1
print('Iteration n*', EM_ITER)

In [ ]:
y_pred = nn.parts['proba'].output(data.X_test)[0]
i = np.random.randint(0, data.X_test.shape[0])
# print('\n'.join('{:1.5f}--{:1.5f}'.format(pred, truth) for pred, truth in zip(y_pred[i], data.y_test[i])))
width = 0.4
plt.bar(np.arange(k_src+k_tgt), y_pred[i], width, color='r', label='pred')
plt.bar(np.arange(k_src+k_tgt)+width, data.y_test[i], width, color='b', label='truth')
# plt.yscale('log')
plt.title("One point distrib")
plt.legend(bbox_to_anchor=(1.2,1.))
plt.show()

[**[EM LOOP]**](#EM-Starts-Here-!)

# ReGenerate Data


[**[Back to top]**](#Introduction)